### Queries on the movie database

In [ ]:
# reading the cleaned data from the stored parquet files in spark dataframes
df_movie = spark.read.parquet(f"{root_path}filtered_data/movies.parquet")
df_lang = spark.read.parquet(f"{root_path}filtered_data/spoken_languages.parquet")
df_cast = spark.read.parquet(f"{root_path}filtered_data/cast.parquet")
df_genre = spark.read.parquet(f"{root_path}filtered_data/genres.parquet")
df_crew = spark.read.parquet(f"{root_path}filtered_data/crew.parquet")
df_recom = spark.read.parquet(f"{root_path}filtered_data/recom.parquet")
df_pcompanies = spark.read.parquet(f"{root_path}filtered_data/production_companies.parquet")

<b> Question 1: </b> \
*Which movie genres have the most movies with a runtime over 120 minutes and how many movies? Please list the top three movie genres.*

In [ ]:
(df_genre
 
     # join df_genre with df_movie
    .join(df_movie, on=df_genre.id==df_movie.id) 
 
     # filter for movies with runtime over 120 minutes
    .where(f.col("runtime")>120)
 
     # count movies per genre
    .groupBy("name")
    .count()
 
     # order results by count in descending order with a limit of 3
    .orderBy("count", ascending=False)
    .limit(3)
    .display())

name,count
Drama,672
Action,298
Thriller,246


<b> Answer: </b> \
The movie genres "Drama", "Action" and "Thriller" have the most movies with a runtime over 120 minutes.

<b> Question 2: </b> \
*In how many movies did the actor Johnny Depp take part in as an actor?*

In [ ]:
jd_movies = (df_cast
     # filtering column name for Johnny Depp and count occurencies
    .where(f.col("name")=="Johnny Depp")
    .count())

print(f"The actor Johnny Depp did take part in {jd_movies} movies as an actor.")

The actor Johnny Depp did take part in 40 movies as an actor.


<b> Answer: </b> \
The actor Johnny Depp did take part in 40 movies as an actor.

*In how many of those did he also act as a producer?*

In [ ]:
(df_cast
     # joining df_cast with df_crew
     .join(df_crew, on=df_cast.id==df_crew.id)
 
     # filter for Johnny Depp appearing as Actor as well as Producer or Executive Producer
     .where(
         (df_cast.name=="Johnny Depp")
          & (df_crew.name=="Johnny Depp")
          & ((df_crew.job=="Producer") | (df_crew.job=="Executive Producer")))
     .count())

Out[20]: 3

<b> Answer: </b> \
Under the assumption that "Executive Producer" also counts as Producer, Johnny Depp acted in 3 movies as a Producer or Executive Producer.

<b> Question 3: </b> \
*List the names and the revenue of the ten movies with the most revenue which were released before 2015.*

In [ ]:
(df_movie
     # filter for release_date before 2015
     .where(
        f.year(f.to_date("release_date", "yyyy-MM-dd"))<2015)
 
     # cast revenue to float 
     .withColumn("revenue", df_movie.revenue.astype(t.FloatType()))
 
     # ordering by revenue and selecting title and revenue with a limit of 10
     .orderBy("revenue", ascending=False)
     .select("title", "revenue")
     .limit(10)
     .display()
)

title,revenue
Avatar,2.78796672E9
Titanic,1.84506394E9
The Avengers,1.51957478E9
Frozen,1.27419878E9
Iron Man 3,1.21544845E9
Transformers: Dark of the Moon,1.12377216E9
The Lord of the Rings: The Return of the King,1.11890253E9
Skyfall,1.10854797E9
Transformers: Age of Extinction,1.0914121E9
The Dark Knight Rises,1.08498726E9


<b> Answer: </b> \
The ten movies listed above are the movies, which were released before 2015 and have the most revenue. The one with best revenue is 'Avatar'.

In [ ]:
(df_movie
     # filter for release_date before 2015
    .where(
        f.year(f.to_date("release_date", "yyyy-MM-dd"))< 2015)
 
     # calculate revenue divided by budget
    .select("title", f.col("revenue")/f.col("budget"), "revenue", "budget")
 
     # ordering after the calculated column
    .orderBy("(revenue / budget)", ascending=False)
    .limit(10)
    .display())

title,(revenue / budget),revenue,budget
House of Flying Daggers,128791.59223300971,92858738,721
The Odd Life of Timothy Green,11654.044933722758,51872154,4451
Shaolin Soccer,11611.181423139598,42752370,3682
Chernobyl Diaries,7810.454506252695,18112444,2319
Girl with a Pearl Earring,7544.356371490281,31437333,4167
Crocodile Dundee II,6990.519738569719,239586083,34273
Garfield: A Tail of Two Kitties,6833.255353009259,141694383,20736
Sympathy for Lady Vengeance,5636.224644549763,23784868,4220
Scooby-Doo 2: Monsters Unleashed,5249.511611765046,181512363,34577
The Blair Witch Project,4872.991316306483,248035258,50900


<b> Answer: </b> \
When comparing the revenue to the budget the ten movies listed above are the financially most succesful. The movie 'House of Flying Daggers' is in this respect the best.

<b> Question 4: </b> \
*What is the movie genre that has a median rating of at least 3 (over all movies with at least ten recommendations) with the lowest average production budget considering all movies?*

In [ ]:
# first getting all the movies with at least 10 recommendations
filtered_movies = (df_recom
                      .groupBy("movie_id")
                      .count()
                      .where(f.col("count")>9))

# filter genres by their median rating
filtered_genres = (df_genre
                      .withColumn("genreName", f.col("name"))
                      .join(df_recom, on=df_genre.id==df_recom.movie_id)
                      .join(filtered_movies, on=filtered_movies.movie_id==df_genre.id)
                      .groupBy("genreName")
                       # calculating the median: 50th percentile corresponds to median
                      .agg(f.expr("percentile_approx(vote, 0.5)").alias("median_rating"))
                      .where(f.col("median_rating")>=3)
                      .select("genreName"))

# join filtered dataframes together and calculate the average budget per genre
(df_genre
    .join(filtered_genres, on=df_genre.name==filtered_genres.genreName)
    .join(df_movie, on=df_genre.id==df_movie.id)
    .groupBy("genreName")
     # calculate the average budget per genre
    .agg(f.avg("budget").alias("avg_budget"))
    .orderBy("avg_budget")
    .limit(1)
    .display())

genreName,avg_budget
Foreign,657476.5294117647


<b> Answer: </b> \
The movie genre 'Foreign' is the genre, which has a median rating of at least 3 with at least 10 recommendations and the lowest average production budget of roughly 657476 considering all movies.

<b> Question 5: </b>\
*How many movies were written by a female writer?*

In [ ]:
# since there could be more than one female writer writing one movie, the id of the movie has to be selected as distinct
(df_crew
    .where(
        (df_crew.job=="Writer") 
         & (df_crew.gender==1))
    .select("id")
    .distinct()
    .count())

Out[6]: 139

In [ ]:
# nicer option with f-string
f_w_movies = df_crew.where((df_crew.job=="Writer") & (df_crew.gender==1)).select("id").distinct().count()
print(f"There are {f_w_movies} movies, which were written by a female writer.")

There are 139 movies, which were written by a female writer.


<b> Answer: </b>\
Under the assumption that '1' in the column gender stands for female (2 for male and 0 for the ones, where the gender is not specified), there are 139 movies, which were written by at least one female writer.

*Explain what data storage structure you used to store the information and why. When storing the information how can you speed up the information retrieval if you know you are interested in looking at the gender of the writer?\
Why does it speed up the information retrieval when you store the data differently?*

<b> Answer: </b>\
For this task I used Apache Parquet Files as data storage structure to store the given information. Apache Parquet is a free and open source, column-oriented data file format designed for efficient data storage and retrieval (definition source: https://parquet.apache.org/). It provides efficient data compression and encoding schemes with enhanced performance to handle complex data in bulk, that's why it's known for both performant data compression and its ability to handle a wide variety of encoding types. Parquet is designed to be a common interchange format for both batch and interactive workloads. It is implemented through using a record shredding and assembly algorithm, which takes into account the complex data structures, that can be used to store the data. When storing data Parquet uses a hybrid model of physical storage layout, which means a combination of columnar and row-wise.\
One of the advantages of Parquet, in contrast to for example CSV, is that column storage files are more lightweight. The reason for that is that in column storage files adequate compression can be done for each column. The column storage architecture allows you to quickly skip data that isn't relevant, which is the reason why queries and aggregations are faster or less time-consuming compared to row-oriented databases. This results in hardware savings, minimizing latency for accessing data as well as saving time and money. Furthermore in each column there are metadata available, which represent for example the minimum, maximum and count of the respective column.\
In addition to that Parquet is best especially for those queries that need to read certain columns from a large table (as processing large volumes of data in the gigabyte range), because Parquet can only read the needed columns therefore greatly minimizing the IO. Parquet also supports nested files and can be compressed as you like, for instance in GZIP, LZO or Snappy. \
So in general Parquet is good for storing big data of any kind (e.g. structured data tables, videos or images), it saves on cloud storage space by using highly efficient column-wise compression and flexible encoding schemes for columns with different data types. Parquet uses different encoding schemes like PLAIN or RLE_DICTIONARY, where the second for example is helpful when having many duplicated and repeated values. This again helps reducing the file size. Furthermore it also supports an increased data throughput and performance using techniques like data skipping, whereby queries that fetch specific column values need not read the entire row of data.\
In summary Parquet is a more efficient data format for bigger files, which is the reason why I used it to store the given movie-information since one of the requirements of this task is to also handle very large amounts of data. \
To speed up the retrieval of specific information you can use the Partitioning of Parquet. With Partitioning you can define how the dataset or rather the data is to be divided into partitions after the columns of the dataset. That means you divide the data into groups (partitions) based on column values, which will improve the performance of queries that restrict results by the partitioned column. For example in this case, when we are looking specifically for the gender of the writer, we could partition after the column 'gender' (e.g. df.write.partitionBy("gender").parquet("parquet-gender")), so that all data with the same gender would be stored in one partition. In this case we could as well partition the dataset after the column 'job', so we would have for instance all writers in one partition. Since in this context we're interested in female writers, a potential Partitioning could be on the column 'gender' and and the column 'job' of the crew dataframe, so the partitioning could look like the following: pf_crew.write.partitionBy("gender", "job").parquet("gender-job-parquet"). But since there are many different jobs in this dataset, the partitioning of the column "job" might not make that a big difference in the performance of the query. So in general Parquet Partitioning speeds up the information retrieval because all the wanted information is stored in partitions next to each other, when partitioning after gender and job, so the query can read all the potential data searching through the whole dataset.